In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import scipy as sp
import scipy.fftpack
import pickle


# Reading data from test.csv

In [2]:
df_test = pd.read_csv("test.csv", low_memory=False)

#appending dataframe into test_Data list
test_Data = []
for i in range(len(df_test)):
    test_Data.append(df_test.iloc[i].values.tolist())

# Creating and Extarcting Features

In [3]:
#Calculation Average
average = []
for i in range(len(test_Data)):
    avg = sum(test_Data[i])/len(test_Data[i])
    average.append(avg/50)

In [4]:
#Calculating diff(max - min)
diff = []
for i in range(len(test_Data)):
    difference = max(test_Data[i]) - min(test_Data[i])
    diff.append(difference/50) 


In [5]:
#Calculating fft values
freq2 = []
freq3 = []
pow2 = []
pow3 = []
for i in range(len(test_Data)):
    temp = test_Data[i]
    fft = sp.fftpack.fft(temp)
    amp = np.abs(fft)
    psd = amp ** 2
    freq = sp.fftpack.fftfreq(len(temp), 1)

    #Sorting and finding max values
    amp_sort = sorted(amp)
    pos_2, pos_3 = amp_sort[-3], amp_sort[-4]

    #checking for peak index by position
    peak_2, peak_3 = np.where(amp == pos_2), np.where(amp == pos_3)

    #getting peak indices
    peak_index2, peak_index3 = peak_2[0][0], peak_3[0][0]

    #peak power values
    pow_2, pow_3 = np.log(psd[peak_index2]), np.log(psd[peak_index3])
    #peak freq values
    freq_2, freq_3 = freq[peak_index2], freq[peak_index3]
        
    #adding into list
    pow2.append(round(pow_2, 3))
    freq2.append(round(freq_2, 3))
    pow3.append(round(pow_3, 3))
    freq3.append(round(freq_3, 3))   
    


In [6]:
#Calculating derivatives
def extractFirstDervative(test_Data):
    firstDervative_list = []
    for i in range(len(test_Data)):
        CGMmax = max(test_Data[i])
        CGMmax_Index = test_Data[i].index(CGMmax)
        diff = np.diff(test_Data[i][0:CGMmax_Index], 1)
        diff = diff.tolist()
        if len(diff) > 0:
            diff_avg = sum(diff)/len(diff)
        else:
            diff_avg = 0
        firstDervative_list.append(diff_avg)
    return firstDervative_list

#Extracting second derivative
def extractSecondDervative(test_Data):
    secondDervative_list = []
    for i in range(len(test_Data)):
        CGMmax = max(test_Data[i])
        CGMmax_Index = test_Data[i].index(CGMmax)
        diff = np.diff(test_Data[i][0:CGMmax_Index], 2)
        diff = diff.tolist()
        if len(diff) > 0:
            diff_avg = sum(diff)/len(diff)
        else:
            diff_avg = 0
        secondDervative_list.append(diff_avg)
    return secondDervative_list

der1, der2 = extractFirstDervative(test_Data), extractSecondDervative(test_Data)


In [9]:
#Create Feature Matrix
data =[]
for i in range(len(test_Data)):
    data.append({
    'Time Difference - Tau' : average[i],
    'CGM difference - dG' : diff[i],
    'Power II' : pow2[i],
    'Frequency II' : freq2[i],
    'Power III' : pow3[i],
    'Frequency III' : freq3[i],
    'First Derivative' : der1[i],
    'Second Derivative' : der2[i]
        })
Data_Feature_Matrix = pd.DataFrame(data)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
Data_Feature_Matrix.iloc[:,:] = scaler.fit_transform(Data_Feature_Matrix.iloc[:,:].to_numpy())

# Loading Pickle Model

In [10]:
cls_PickleModel = pickle.load(open("RandomForestClassifier.pkl", "rb"))
pickle_Results = cls_PickleModel.predict(Data_Feature_Matrix)

# Extracting Results

In [11]:
pd.DataFrame(pickle_Results).to_csv('Result.csv',index=False,header=False)